# Dashboard Prototype
- ideas of visualizatoins we can use to report Omicron variant

In [8]:
#import packages
#Import Packages necessary for anlaysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import datetime
from datetime import date, timedelta
import sys
import seaborn as sns

import requests
import os

#used for formatting visualizations
%matplotlib inline

In [12]:
#look at OWID case data
#check to see if this data discrepancy is present in OWID (COVID-19 cases) raw data:
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
df_OWID = pd.read_csv(url)
print (df_OWID.dtypes)
df_OWID.tail()

iso_code                                    object
continent                                   object
location                                    object
date                                        object
total_cases                                float64
                                            ...   
human_development_index                    float64
excess_mortality_cumulative_absolute       float64
excess_mortality_cumulative                float64
excess_mortality                           float64
excess_mortality_cumulative_per_million    float64
Length: 67, dtype: object


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
137391,ZWE,Africa,Zimbabwe,2021-11-28,133951.0,115.0,43.429,4705.0,1.0,0.857,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
137392,ZWE,Africa,Zimbabwe,2021-11-29,134226.0,275.0,78.857,4706.0,1.0,1.000,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
137393,ZWE,Africa,Zimbabwe,2021-11-30,134625.0,399.0,135.857,4707.0,1.0,1.143,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
137394,ZWE,Africa,Zimbabwe,2021-12-01,135337.0,712.0,227.143,4707.0,0.0,0.571,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
137395,ZWE,Africa,Zimbabwe,2021-12-02,136379.0,1042.0,372.143,4707.0,0.0,0.429,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


In [13]:
#changing the date-types for date
df_OWID['date'] = pd.to_datetime(df_OWID['date'])
df_OWID.dtypes

iso_code                                           object
continent                                          object
location                                           object
date                                       datetime64[ns]
total_cases                                       float64
                                                ...      
human_development_index                           float64
excess_mortality_cumulative_absolute              float64
excess_mortality_cumulative                       float64
excess_mortality                                  float64
excess_mortality_cumulative_per_million           float64
Length: 67, dtype: object

In [14]:
#observe case data for 12.2.2021
#Subset to June, 2021

start_date = "2021-12-02"
end_date = "2021-12-02"

#sub-analysis of specific dates, building off of map data "no_VI"
after_start_date = df_OWID["date"] >= start_date
before_end_date = df_OWID["date"] <= end_date
between_two_dates = after_start_date & before_end_date
df_Dec2 = df_OWID.loc[between_two_dates]

print("Num of Total Obs.", df_Dec2.date.unique())

Num of Total Obs. ['2021-12-02T00:00:00.000000000']


In [71]:
#extract metrics of interests

#only keeping necessary variables from this dataset, for our observations
df_Dec2_cases = df_Dec2[['location', 'iso_code',"date", "total_cases", 'new_cases', 
                          'total_deaths', 'new_deaths']]

#renaming variables
df_Dec2_cases.rename(columns={ 'location': 'country'}, inplace = True)
df_Dec2_cases.rename(columns={ 'new_cases': 'new_covid_cases'}, inplace = True)
df_Dec2_cases.rename(columns={ 'total_cases': 'total_covid_cases'}, inplace = True)
df_Dec2_cases.head()

C:\Users\bthrift\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,country,iso_code,date,total_covid_cases,new_covid_cases,total_deaths,new_deaths
647,Afghanistan,AFG,2021-12-02,157387.0,28.0,7310.0,1.0
1306,Africa,OWID_AFR,2021-12-02,8681372.0,15939.0,223189.0,152.0
1953,Albania,ALB,2021-12-02,200639.0,466.0,3104.0,3.0
2600,Algeria,DZA,2021-12-02,210921.0,198.0,6083.0,7.0
3241,Andorra,AND,2021-12-02,17658.0,232.0,132.0,1.0


In [72]:
#Investigate weird Data
Where_GBR = df_OWID.loc[(df_OWID['iso_code'] == 'GBR')]
Where_GBR.head(2)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
128986,GBR,Europe,United Kingdom,2020-01-31,2.0,2.0,NaN,NaN,NaN,NaN,...,20.0,24.7,NaN,2.54,81.32,0.932,NaN,NaN,NaN,NaN
128987,GBR,Europe,United Kingdom,2020-02-01,2.0,0.0,NaN,NaN,NaN,NaN,...,20.0,24.7,NaN,2.54,81.32,0.932,NaN,NaN,NaN,NaN


In [73]:
##extract countries of interests
Omicron_Dec2_countries = df_Dec2_cases [df_Dec2_cases.country.isin([
    'Australia', 'Austria','Belgium', 'Botswana', 'Brazil', 'Canada', 'Czechia',
    'United Kingdom', 'Germany', 'Ghana', 'Hong Kong', 'Ireland', 'Israel', 'Italy', 
    'Japan', 'Netherlands', 'Portugal', 'Reunion', 'South Africa', 'Spain', 'Sweden'    
])]
print (Omicron_Dec2_countries.country.nunique())
print (Omicron_Dec2_countries.country.unique())
Omicron_Dec2_countries.head()
#Reunion a a country is not represented in OWID

20
['Australia' 'Austria' 'Belgium' 'Botswana' 'Brazil' 'Canada' 'Czechia'
 'Germany' 'Ghana' 'Hong Kong' 'Ireland' 'Israel' 'Italy' 'Japan'
 'Netherlands' 'Portugal' 'South Africa' 'Spain' 'Sweden' 'United Kingdom']


,country,iso_code,date,total_covid_cases,new_covid_cases,total_deaths,new_deaths
7746,Australia,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0
8393,Austria,AUT,2021-12-02,1179244.0,8882.0,12625.0,72.0
12888,Belgium,BEL,2021-12-02,1809557.0,23113.0,27120.0,48.0
17101,Botswana,BWA,2021-12-02,195302.0,234.0,2419.0,1.0
17747,Brazil,BRA,2021-12-02,22118782.0,12910.0,615179.0,215.0


In [31]:
#read in GISAID metadata
#processed from PTC processing script
#GISAID data last processed 11.22.2021

df_metadata = pd.read_csv('C:\\Users\\bthrift\\Documents\\Projects\\NGS Map\\R_script_required_data\\GISAID_processed_metadata_11_24_2021.csv')
df_metadata.head()

C:\Users\bthrift\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,197,198,203) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,gisaid_collect_date,gisaid_country,All lineages,AY.1,AY.10,AY.100,AY.101,AY.102,AY.103,AY.104,...,gisaid_collect_yearweek,gisaid_collect_weekstartdate,who_region,aggregate_location,seq_count,gisaid_lagdays_median,gisaid_lagdays_min,gisaid_lagdays_max,gisaid_lagdays_q1,gisaid_lagdays_q3
0,2019-12-24,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-W52,2019-12-23,Eastern Mediterranean,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-12-24,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-W52,2019-12-23,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-12-24,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-W52,2019-12-23,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-24,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-W52,2019-12-23,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-24,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-W52,2019-12-23,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
#extract countries of interests
##extract countries of interests
Omicron_countries = df_metadata [df_metadata.gisaid_country.isin([
    'Australia', 'Austria','Belgium', 'Botswana', 'Brazil', 'Canada', 'Czechia',
    'United Kingdom', 'Germany', 'Ghana', 'Hong Kong', 'Ireland', 'Israel', 
    'Italy', 'Japan', 'Netherlands', 'Portugal', 'Reunion', 'South Africa', 'Spain', 'Sweden'    
])]
print (Omicron_countries.gisaid_country.nunique())
print (Omicron_countries.gisaid_country.unique())
Omicron_countries.tail()
#Reunion IS represented in GISAID data

21
['Australia' 'Austria' 'Belgium' 'Botswana' 'Brazil' 'Canada' 'Czechia'
 'Germany' 'Ghana' 'Hong Kong' 'Ireland' 'Israel' 'Italy' 'Japan'
 'Netherlands' 'Portugal' 'South Africa' 'Spain' 'Sweden' 'United Kingdom'
 'Reunion']


,gisaid_collect_date,gisaid_country,All lineages,AY.1,AY.10,AY.100,AY.101,AY.102,AY.103,AY.104,...,gisaid_collect_yearweek,gisaid_collect_weekstartdate,who_region,aggregate_location,seq_count,gisaid_lagdays_median,gisaid_lagdays_min,gisaid_lagdays_max,gisaid_lagdays_q1,gisaid_lagdays_q3
168503,2021-11-14,South Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021-W45,2021-11-08,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168506,2021-11-14,Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021-W45,2021-11-08,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168510,2021-11-14,Sweden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021-W45,2021-11-08,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168530,2021-11-14,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021-W45,2021-11-08,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168552,2021-11-14,Reunion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021-W45,2021-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
#extract WHO region name
#only keeping necessary variables from this dataset, for our observations
Omicron_countries_region = Omicron_countries[['gisaid_country', "who_region"]]

#renaming variables
Omicron_countries_region.rename(columns={ 'gisaid_country': 'country'}, inplace = True)
Omicron_countries_region.head()

C:\Users\bthrift\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,country,who_region
10,Australia,Western Pacific
11,Austria,Europe
18,Belgium,Europe
26,Botswana,Africa
27,Brazil,Americas


In [45]:
# initialize list of lists
data = [['Australia', 9], ['Austria', 4], ['Belgium', 2],
       ['Botswana', 19], ['Brazil', 3], ['Canada', 3],
       ['Czech Republic', 1], ['England', 18], ['Germany', 5],
        ['Ghana', 33], ['Hong Kong', 8], ['Ireland', 1],
        ['Israel', 1], ['Italy', 4], ['Japan', 2],
        ['Netherlands', 13], ['Portugal', 13], ['Reunion', 1],
        ['South Africa', 172], ['Spain', 2], ['Sweden', 1]
       ]

  
# Create the pandas DataFrame
df_Omic_cases = pd.DataFrame(data, columns = ['country', 'omicron_variant'])
  
# print dataframe.
df_Omic_cases.head()

,country,omicron_variant
0,Australia,9
1,Austria,4
2,Belgium,2
3,Botswana,19
4,Brazil,3


In [74]:
#merge GISAID and OWID
Omicron_cases_regions = Omicron_Dec2_countries.merge(Omicron_countries_region, on = 'country')
Omicron_cases_regions.head()

,country,iso_code,date,total_covid_cases,new_covid_cases,total_deaths,new_deaths,who_region
0,Australia,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific
1,Australia,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific
2,Australia,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific
3,Australia,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific
4,Australia,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific


In [75]:
#merge GISAID and OWID
Omicron_dashboard = df_Omic_cases.merge(Omicron_cases_regions, on = 'country')
Omicron_dashboard.head()

,country,omicron_variant,iso_code,date,total_covid_cases,new_covid_cases,total_deaths,new_deaths,who_region
0,Australia,9,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific
1,Australia,9,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific
2,Australia,9,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific
3,Australia,9,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific
4,Australia,9,AUS,2021-12-02,214879.0,1522.0,2033.0,12.0,Western Pacific


In [76]:
#export final dataset with country, region, case, date
Omicron_dashboard = Omicron_dashboard[['country', 'iso_code',"who_region", 'omicron_variant',
                                      'new_covid_cases', 'new_deaths', 'total_covid_cases', 
                                       'total_deaths','date']]
print (Omicron_dashboard.country.nunique())
print (Omicron_dashboard.shape)
Omicron_dashboard.head()

18
(12384, 9)


,country,iso_code,who_region,omicron_variant,new_covid_cases,new_deaths,total_covid_cases,total_deaths,date
0,Australia,AUS,Western Pacific,9,1522.0,12.0,214879.0,2033.0,2021-12-02
1,Australia,AUS,Western Pacific,9,1522.0,12.0,214879.0,2033.0,2021-12-02
2,Australia,AUS,Western Pacific,9,1522.0,12.0,214879.0,2033.0,2021-12-02
3,Australia,AUS,Western Pacific,9,1522.0,12.0,214879.0,2033.0,2021-12-02
4,Australia,AUS,Western Pacific,9,1522.0,12.0,214879.0,2033.0,2021-12-02


In [77]:
#drop duplicates
Omicron_dashboard.drop_duplicates(subset = ['country', 'iso_code', 'date'], inplace=True)
Omicron_dashboard.shape

(18, 9)

In [78]:
Omicron_dashboard.head(20)
Omicron_dashboard.sort_values(by='omicron_variant', ascending=False, inplace=True)
Omicron_dashboard.head(4)
Omicron_dashboard.head(20)

,country,iso_code,who_region,omicron_variant,new_covid_cases,new_deaths,total_covid_cases,total_deaths,date
10320,South Africa,ZAF,Africa,172,11535.0,44.0,2988148.0,89915.0,2021-12-02
4816,Ghana,GHA,Africa,33,0.0,0.0,130920.0,1209.0,2021-12-02
2064,Botswana,BWA,Africa,19,234.0,1.0,195302.0,2419.0,2021-12-02
8944,Netherlands,NLD,Europe,13,23104.0,65.0,2726156.0,19974.0,2021-12-02
9632,Portugal,PRT,Europe,13,2898.0,13.0,1154817.0,18471.0,2021-12-02
0,Australia,AUS,Western Pacific,9,1522.0,12.0,214879.0,2033.0,2021-12-02
5504,Hong Kong,HKG,NaN,8,6.0,0.0,12445.0,213.0,2021-12-02
4128,Germany,DEU,Europe,5,73279.0,390.0,6072299.0,102573.0,2021-12-02
688,Austria,AUT,Europe,4,8882.0,72.0,1179244.0,12625.0,2021-12-02
7568,Italy,ITA,Europe,4,16810.0,72.0,5060430.0,134003.0,2021-12-02


# Aggregate data by WHO Region

In [65]:
#aggregation by date and country
Omicron_variant_region = Omicron_dashboard.groupby([ 'date', 'who_region'])['omicron_variant'].sum().reset_index()
Omicron_variant_region.head()

,date,who_region,omicron_variant
0,2021-12-02,Africa,224
1,2021-12-02,Americas,6
2,2021-12-02,Europe,46
3,2021-12-02,Western Pacific,11


In [69]:
#import geometric polygons extracted from Flourish.com (textfile)
df_file_geo = pd.read_table ("C:\\Users\\bthrift\\Documents\\Projects\\NGS Map\\NGS underlying data\\Flourish\\geometry_flourish.txt", sep='\t')
df_geometry = df_file_geo[['code', "geometry"]]
df_geometry.head(2)

,code,geometry
0,NaN,"{""type"":""Polygon"",""coordinates"":[[[28.04511,9...."
1,NaN,"{""type"":""Polygon"",""coordinates"":[[[28.04511,9...."


In [80]:
#Drop countries with NaN country_code & duplicates
df_geometry_test = df_geometry[df_geometry['code'].notna()]
df_geometry_test.drop_duplicates(subset = ['code'], inplace=True)
df_geometry_test.rename(columns={ 'code': 'iso_code'}, inplace = True)
df_geometry_test.nunique()

<ipython-input-80-b5edd9601325>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geometry_test.drop_duplicates(subset = ['code'], inplace=True)
C:\Users\bthrift\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


iso_code    237
geometry    236
dtype: int64

In [82]:
#merge together dataset
Omicron_geo = Omicron_dashboard.merge(df_geometry_test, how = 'left', on = 'iso_code')
Omicron_geo.head(2)

,country,iso_code,who_region,omicron_variant,new_covid_cases,new_deaths,total_covid_cases,total_deaths,date,geometry
0,South Africa,ZAF,Africa,172,11535.0,44.0,2988148.0,89915.0,2021-12-02,"{""type"":""MultiPolygon"",""coordinates"":[[[[16.48..."
1,Ghana,GHA,Africa,33,0.0,0.0,130920.0,1209.0,2021-12-02,"{""type"":""Polygon"",""coordinates"":[[[-2.766622,5..."


In [83]:
Omicron_geo.to_csv('C:\\Users\\bthrift\\Documents\\Projects\\Omicron Variant Report\\data\\Omicron_dashboard_geo_df.csv', 
                  index=True, sep=',')